In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.remote.webelement import WebElement

options = Options()
# 取消彈出式視窗，以免影響爬蟲作業
options.add_argument('--disable-notifications')

# TGOS地理資訊整合圖台
driver = webdriver.Chrome(options = options)
driver.get('https://map.tgos.tw/TGOSCloudMap?token=#')

driver.implicitly_wait(5)

# 點擊彈出視窗的"關閉"鈕
addr = driver.find_element(By.CSS_SELECTOR, '.btn.btn-danger.bootbox-accept')
addr.click()

# 點旁邊三條線
addr1 = driver.find_element(By.CSS_SELECTOR, '.leaflet-control-expand.text-dark')
addr1.click()

# 輸入欲查詢地址
addr2 = driver.find_element(By.ID, 'locate2DAddr')
addr2.send_keys('臺北市信義區市府路45號')

# 按下搜尋鈕
result = driver.find_element(By.CSS_SELECTOR,'.btn.btn-primary')
result.click()

# 獲取經緯度信息
time.sleep(5)

geocoding_info = driver.find_elements(By.CSS_SELECTOR, "li[class = 'list-group-item  list-group-item-action addr-locate-item py-3']")
print(geocoding_info)

def get_header(ele: WebElement):
    sub_ele = ele.find_element(By.CSS_SELECTOR, 'div:nth-child(1)')
    return sub_ele.text
    
def get_value(ele: WebElement):
    sub_ele = ele.find_element(By.CSS_SELECTOR, 'div:nth-child(2)')
    return sub_ele.text

headers = list(map(get_header, geocoding_info))
values = list(map(get_value, geocoding_info))

print(headers)
print(values)

driver.close()

import pandas as pd

data = {'地址': headers,
       '行政區域': values}

df = pd.DataFrame(data)
df['行政區'] = df['行政區域'].str.extract(r'([一-龥]+(?:區|市|鎮|鄉))')
df['geometry'] = df['行政區域'].str.findall(r'(\d+\.?\d*)')
# 刪掉行政區域欄
df = df.drop(['行政區域'], axis = 1)

print(df)

[<selenium.webdriver.remote.webelement.WebElement (session="de6d97c4cae551d20831eed307f4b076", element="f.42649E1A222E64126FF1C566249AAE25.d.6173E67BFE13ABBED7FD35B1BEDC604D.e.98")>]
['臺北市信義區西村里8鄰市府路45號']
['臺北市信義區25.034026,121.563903']
                  地址     行政區                 geometry
0  臺北市信義區西村里8鄰市府路45號  臺北市信義區  [25.034026, 121.563903]


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

from dotenv import load_dotenv
load_dotenv()

import os

OpenTrip_API = os.environ.get('OpenTripMap_API_key')

import requests


BASE_URL = 'https://api.opentripmap.com/0.1/en/places/radius'

params = {
    'radius': 1000,
    'lon': 121.4022,
    'lat': 24.5915,
    'apikey': OpenTrip_API,
    'kinds': 'amusement_parks'
}

response = requests.get(BASE_URL, params = params)
time.sleep(5)

'''
新北市中心座標
lon = 121.4022
lat = 24.5915
'''
print('Status code:', response.status_code)
print('Response JSON:', response.json())

Status code: 200
Response JSON: {'type': 'FeatureCollection', 'features': []}


In [1]:
import requests
from pprint import pprint
import time
import json

from dotenv import load_dotenv
load_dotenv()

import os

TDX_API_ID = os.environ.get('TDX_APP_ID')
TDX_API_KEY = os.environ.get('TDX_API_KEY')

app_id = TDX_API_ID
app_key = TDX_API_KEY

auth_url = 'https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token'
url = 'https://tdx.transportdata.tw/api/tourism/service/odata/V2/Tourism/Attraction/'

class Auth():
    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        content_type = 'application/x-www-form-urlencoded'
        grant_type = 'client_credentials'

        return{
            'content_type': content_type,
            'grant_type': grant_type,
            'client_id': self.app_id,
            'client_secret': self.app_key
        }

class data():
    def __init__(self, app_id, app_key, auth_response):
        self.app_id = app_id
        self.app_key = app_key
        self.auth_response = auth_response

    def get_data_header(self):
        auth_JSON = json.loads(self.auth_response.text)
        access_token = auth_JSON.get('access_token')

        return{
            'authorization' : 'Bearer ' + access_token,
            'accept-Encoding' : 'gzip'
        }

if __name__ == '__main__':
    try:
        d = data(app_id, app_key, auth_response)
        data_response = requests.get(url, headers = d.get_data_header())
    except:
        a = Auth(app_id, app_key)
        auth_response = requests.post(auth_url, a.get_auth_header())
        d = data(app_id, app_key, auth_response)
        data_response = requests.get(url, headers = d.get_data_header())
    print(auth_response)
    pprint(auth_response.text)
    print(data_response)
    pprint(data_response.text)

<Response [200]>
('{"access_token":"eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJER2lKNFE5bFg4WldFajlNNEE2amFVNm9JOGJVQ3RYWGV6OFdZVzh3ZkhrIn0.eyJleHAiOjE3NjEyMzA3MzEsImlhdCI6MTc2MTE0NDMzMSwianRpIjoiOTM4MDliMjktZjJiMi00NWJiLWI0NzUtZmRkYjE4NTIwYmYwIiwiaXNzIjoiaHR0cHM6Ly90ZHgudHJhbnNwb3J0ZGF0YS50dy9hdXRoL3JlYWxtcy9URFhDb25uZWN0Iiwic3ViIjoiYjg0ZWZjOGMtM2JlOS00ZjgzLWExMzUtNDQ3YzYxYWIzNWZiIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiY3dlbmhzdWFuLTdmZDhjNjJhLTIwNzItNGIxNiIsImFjciI6IjEiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsic3RhdGlzdGljIiwicHJlbWl1bSIsInBhcmtpbmdGZWUiLCJtYWFzIiwiYWR2YW5jZWQiLCJnZW9pbmZvIiwidmFsaWRhdG9yIiwidG91cmlzbSIsImhpc3RvcmljYWwiLCJjd2EiLCJiYXNpYyJdfSwic2NvcGUiOiJwcm9maWxlIGVtYWlsIiwidXNlciI6IjI3MjVhYWMwIn0.pH6shAlkNv35hfspWdeRmGyDK_0rrC7SleZrFqys5quZ6cf77Jp0z8GfaO8j4hROHnT9vDmK6RqOAyyoyC52czrKKc2Zre-UUWSABtEDgnzAd6mIsxAjWHYvh4fNLPxW7Aw6R5zEY_qx-WBpuVHsi0kg2T8aGDqm6Po6nvjKfgJQV6AMwHROLGEpiFjO-LJ-2ct0L1YQ9VesNKyoeQaTKicJcjpe0po8MOA_lv5-cESPh40k9SyyfQ7HK_Kof6SNsZX6v_2mU9f1PILpO1LC4LlI